### NLP里的迁移学习
> * 使用预训练好的模型来抽取词、句子特征
>> * 例如word2vec或语言模型
> * 不更新预训练好的模型
> * 需要构建新的网络来抓取新任务需要的信息
>> * Word2vec忽略了时序信息，语言模型只看了一个方向

### BERT的动机
> * 基于微调的NLP模型
> * 预训练的模型抽取了足够多的信息
> * 新的任务只需要增加一个简单的输出层

### BERT架构
> * 只有编码器的Transformer
> * 两个版本：
>> * Base：#block=12,hiddens_size=768,#heads=12,#parameters=110M
>> * Large：#block=24,hiddens_size=1024,#heads=16,#parameters=340M
> * 在大规模数据上训练>3B(30亿)词

### 对输入的修改
> * 每个样本是一个句子对
> * 加入额外的片段嵌入
> * 位置编码可学习
> ![](../images/BERT-Input.png)

### 预训练任务1：带掩码的语言模型
> * Transformer的编码器是双向的，标准语言模型要求单向
> * 带掩码的语言模型每次随机（15%概率）将一些词元换成<mask>
> * 因为微调任务重不出现<mask>
>> * 80%概率下，将选中的词元变成<mask>
>> * 10%概率下换成一个随机词元
>> * 10%概率下保持原有词元

### 预训练任务2：下一个句子预测
> * 预测一个句子对中两个句子是不是相邻
> * 训练样本中：
>> * 50%概率选择相邻句子对：\<cls> this movie is great \<sep> i like it \<sep>
>> * 50%概率选择随机句子对：\<cls> this movie is great \<sep> hello world \<sep>
> * 将\<cls>对应的输出放到一个全连接层来预测

### 总结
> * BERT针对微调设计
> * 基于Transformer的编码器做了如下修改
>> * 模型更大，训练数据更多
>> * 输入句子对，片段嵌入，可学习的位置编码
>> * 训练时使用两个任务：
>>> * 带掩码的语言模型
>>> * 下一个句子预测

In [1]:
import os
import random
import torch
from torch import nn
from d2l import limutorch as d2l

In [2]:
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """得到BERT输入的句子和他们的分隔符"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

In [3]:
class BERTEncoder(nn.Module):
    """BERT编码器"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, max_len=1000, key_size=768, query_size=768,
                 value_size=768, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}",
                                 d2l.EncoderBlock(key_size, query_size, value_size, num_hiddens,
                                                  norm_shape, ffn_num_input, ffn_num_hiddens,
                                                  num_heads, dropout, True))
        # 在BERT中，位置嵌入式可学习的，因此创建一个足够长的位置嵌入参数
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # 在以下代码中X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

In [4]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

In [5]:
tokens = torch.randint(0, vocab_size, (2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoder_X = encoder(tokens, segments, None)
tokens, encoder_X.shape

(tensor([[ 819, 7175, 4273, 8130, 5426, 4584, 1591, 1576],
         [8349, 7705, 7663, 8608, 3376, 6951, 3152, 9338]]),
 torch.Size([2, 8, 768]))

In [6]:
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))

    """
        这里的pred_positions是在告知句子中哪些位置被mask了，需要预测，程序将根据这些位置获取BERT中抽取的特征进行预测，
        这里的X就是BERT抽取的特征
    """
    def forward(self, X, pred_positions):
        print(X.shape)
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        print(pred_positions.shape)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # 假设batch_size=2， num_pred_positions=3
        # 那么batch_idx为np.array([0, 0, 0, 1, 1, 1])
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        print(batch_idx.shape)
        masked_X = X[batch_idx, pred_positions]
        print(masked_X.shape)
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

In [7]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoder_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 8, 768])
torch.Size([6])
torch.Size([6])
torch.Size([6, 768])


torch.Size([2, 3, 10000])

In [8]:
X = torch.arange(160).reshape(2, 10, 8)
positions = torch.tensor([[1, 5, 2], [6, 1, 5]]).reshape(-1)
batch_idx = torch.arange(0, 2)
batch_idx = torch.repeat_interleave(batch_idx, 3)
masked_X = X[batch_idx, positions]
X, batch_idx, masked_X, masked_X.shape

(tensor([[[  0,   1,   2,   3,   4,   5,   6,   7],
          [  8,   9,  10,  11,  12,  13,  14,  15],
          [ 16,  17,  18,  19,  20,  21,  22,  23],
          [ 24,  25,  26,  27,  28,  29,  30,  31],
          [ 32,  33,  34,  35,  36,  37,  38,  39],
          [ 40,  41,  42,  43,  44,  45,  46,  47],
          [ 48,  49,  50,  51,  52,  53,  54,  55],
          [ 56,  57,  58,  59,  60,  61,  62,  63],
          [ 64,  65,  66,  67,  68,  69,  70,  71],
          [ 72,  73,  74,  75,  76,  77,  78,  79]],
 
         [[ 80,  81,  82,  83,  84,  85,  86,  87],
          [ 88,  89,  90,  91,  92,  93,  94,  95],
          [ 96,  97,  98,  99, 100, 101, 102, 103],
          [104, 105, 106, 107, 108, 109, 110, 111],
          [112, 113, 114, 115, 116, 117, 118, 119],
          [120, 121, 122, 123, 124, 125, 126, 127],
          [128, 129, 130, 131, 132, 133, 134, 135],
          [136, 137, 138, 139, 140, 141, 142, 143],
          [144, 145, 146, 147, 148, 149, 150, 151],
         

In [9]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

In [10]:
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 2)

    def forward(self, X):
        # X的形状为（batch_size,num_hiddens）
        return self.output(X)

In [11]:
"""NextSentencePred实例的欠下哪个判断返回每个BERT输入序列的二分类预测"""
encoder_X = torch.flatten(encoder_X, start_dim=1)
print(encoder_X.shape)
# NSP的输入形状为（batch_size,num_hiddens）
nsp = NextSentencePred(encoder_X.shape[-1])
nsp_Y_hat = nsp(encoder_X)
nsp_Y_hat.shape

torch.Size([2, 6144])


torch.Size([2, 2])

In [12]:
"""计算两个二元分类的交叉熵损失"""
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

In [13]:
class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 num_layers, dropout, maxlen=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768, nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                                   num_heads, num_layers, dropout, max_len=maxlen, key_size=key_size,
                                   query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None, pred_positions=None):
        encoder_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoder_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类的隐藏层，0是“<cls>”标记的索引
        nsp_Y_hat = self.nsp(self.hidden(encoder_X[:, 0, :]))
        return encoder_X, mlm_Y_hat, nsp_Y_hat

In [14]:
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

In [15]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        """
            random.choice()会从非空序列中随机选取一个数据返回，可以是list，tuple，str，set
            random.choice('choice')会返回choice中的任意一个字母，可能是c，也可能是h或者其他
        """
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

In [16]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    """生成下一句预测任务的数据"""
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(paragraph[i], paragraph[i+1], paragraphs)
        # 考虑1个’<cls>‘词元和2ge’<sep>‘词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

In [17]:
"""
    tokens是表示BERT输入序列的词元的列表
    candidate_pred_positions是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测）
    num_mlm_preds指示预测的数量（选择15%要预测的随机词元）
"""
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    """生成遮蔽语言模型任务的数据"""
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的’<mask>‘或者随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间，将词替换成"<mask>"词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%时间，保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%时间，用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position])
        )
    # 返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签
    return mlm_input_tokens, pred_positions_and_labels

In [18]:
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens, candidate_pred_positions,
                                                                      num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x:x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

In [19]:
"""
    定义以下函数将特殊的“<mask>”词元附加到输入中
    参数examples包含来自两个训练任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出
"""
def _pad_bert_inputs(examples, max_len, vocab):
    """将文本转换为预训练数据集"""
    max_num_mlm_preds = round(max_len * 0.15)
    all_tokens_ids, all_segments, valid_lens = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        all_tokens_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)),
                                           dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括’<pad>‘计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions +
                                               [0] * (max_num_mlm_preds - len(pred_positions)),
                                               dtype=torch.long))
        # 填充词元的预测将通过乘以0在损失中过滤掉
        all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) +
                                            [0.0] * (max_num_mlm_preds - len(pred_positions)),
                                            dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids +
                                           [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)),
                                           dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
        return (all_tokens_ids, all_segments, valid_lens, all_pred_positions,
                all_mlm_weights, all_mlm_labels, nsp_labels)

In [20]:
class _WikiTextDate(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表
        # 而输出paragraphs[i]是代表断乱的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5,
                               reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab) + (segments, is_next))
                    for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx],self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

In [21]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDate(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

In [22]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, nsp_y.shape)
    break

In [23]:
len(vocab)

20256

In [24]:
net = BERTModel(len(vocab), num_hiddens=128, norm_shape=[128], ffn_num_input=128, ffn_num_hiddens=256,
                num_heads=2, num_layers=2, dropout=0.2, key_size=128, query_size=128, value_size=128,
                hid_in_features=128, mlm_in_features=128, nsp_in_features=128)
devices = d2l.try_all_gpus()
loss = nn.CrossEntropyLoss()

In [25]:
def _get_batch_loss_bert(net, loss, vocab_size, tokens_X, segments_X, valid_lens_x, pred_positions_X,
                         mlm_weights_X, mlm_Y, nsp_y):
    # 前向传播
    _, mlm_Y_hat, nsp_Y_hat = net (tokens_X, segments_X, valid_lens_x.reshape(-1), pred_positions_X)

    # 计算遮蔽语言模型损失
    mlm_l = loss(mlm_Y_hat.reshape(-1, vocab_size), mlm_Y.reshape(-1)) * mlm_weights_X.reshape(-1, 1)
    mlm_l = mlm_l.sum() / (mlm_weights_X.sum() + 1e-8)
    # 计算下一句子预测任务的损失
    nsp_l = loss(nsp_Y_hat, nsp_y)
    l = mlm_l + nsp_l
    return mlm_l, nsp_l, l

In [26]:
def train_bert(train_iter, net, loss, vocab_size, devices, num_steps):
    net = nn.DataParallel(net, device_ids = devices).to(devices[0])
    trainer = torch.optim.Adam(net.parameters(), lr=0.01)
    step, timer = 0, d2l.Timer()
    animator = d2l.Animator(xlabel='step', ylabel='loss', xlim=[1, num_steps], legend=['mlm', 'nsp'])
    # 遮蔽语言模型损失的和，下一句预测任务损失的和，句子对的数量，计数
    metric = d2l.Accumulator(4)
    num_steps_reached = False
    while step < num_steps and not num_steps_reached:
        for tokens_X, segments_X, valid_lens_x, pred_positions_X,\
            mlm_weights_X, mlm_Y, nsp_y in train_iter:
            tokens_X = tokens_X.to(devices[0])
            segments_X = segments_X.to(devices[0])
            valid_lens_x = valid_lens_x.to(devices[0])
            pred_positions_X = pred_positions_X.to(devices[0])
            mlm_weights_X = mlm_weights_X.to(devices[0])
            mlm_Y, nsp_y = mlm_Y.to(devices[0]), nsp_y.to(devices[0])
            trainer.zero_grad()
            timer.start()
            mlm_l, nsp_l, l = _get_batch_loss_bert(net, loss, vocab_size, tokens_X, segments_X,
                                                   valid_lens_x, pred_positions_X, mlm_weights_X,
                                                   mlm_Y, nsp_y)
            l.backward()
            trainer.step()
            metric.add(mlm_l, nsp_l, tokens_X.shape[0], 1)
            timer.stop()
            animator.add(step + 1, (metric[0] / metric[3], metric[1]/ metric[3]))
            step += 1
            if step == num_steps:
                num_steps_reached = True
                break

    print(f'MLM loss {metric[0] / metric[3]:.3f}, NSP loss {metric[1] / metric[3]:.3f}')
    print(f'{metric[2] / timer.sum():.1f} sentence pairs/sec on {str(devices)}')

In [ ]:
train_bert(train_iter, net, loss, len(vocab), devices, 50)

In [ ]:
def get_bert_encoding(net, tokens_a, tokens_b = None):
    tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
    token_ids = torch.tensor(vocab[tokens], device=devices[0]).unsqueeze(0)
    segments =  torch.tensor(segments, device=devices[0]).unsqueeze(0)
    valid_len = torch.tensor(len(tokens), device=devices[0]).unsqueeze(0)
    encoded_X, _, _ = net(token_ids, segments, valid_len)
    return encoder_X

In [ ]:
tokens_a = ['a', 'crane', 'is', 'flying']
encoded_text = get_bert_encoding(net, tokens_a)
# 词元：'<cls>','a','crane','is','flying','<sep>'
# “<cls>”（用于分类）和“<sep>”（用于分隔）
# 因为零是“<cls>”词元，encoded_text[:, 0, :]是整个输入语句的BERT表示
encoded_text_cls = encoded_text[:, 0, :]
encoded_text_crane = encoded_text[:, 2, :]
encoded_text.shape, encoded_text_cls.shape, encoded_text_crane[0][:3]

In [ ]:
tokens_a, tokens_b = ['a', 'crane', 'driver', 'came'], ['he', 'just', 'left']
encoded_pair = get_bert_encoding(net, tokens_a, tokens_b)
# 词元：'<cls>','a','crane','driver','came','<sep>','he','just', 'left','<sep>'
encoded_pair_cls = encoded_pair[:, 0, :]
encoded_pair_crane = encoded_pair[:, 2, :]
encoded_pair.shape, encoded_pair_cls.shape, encoded_pair_crane[0][:3]